# Quantum Support Vector Machine (QSVM)

QSVM is a Quantum Machine Learning (QML) algorithm designed for binary classification, inspired by the classical Support Vector Machine (SVM). In the following, we will briefly review the SVM algorithm and then study its quantum version. After that, you will be ready to apply the QSVM to two example datasets from Scikit-learn.

### 1. Overview of SVM algorithm

Let's suppose that we are given a set of points $x \in \mathbb{R}^2$ lying on a plane, each of them with a label $y \in \{0, 1\}$. The goal is to find a line that separates the points of different labels (see Figure below, taken from [3], where the color indicates the binary label). As you may have noticed, if such a line exists, which sometimes does not, there are infinite lines that do the job. However, some are better than others, as they have a greater margin from the closest points from each class.

![Separating line](figures_qsvm/svm_separating_lines.png)
![Margin lines](figures_qsvm/svm_margin_hyperplanes.png)


So far we have introduced a linear classifier, but the SVM is more than that, it is a ML algorithm (see [1] for an informal introduction to ML or [2] if you are interested in delving more into it). Thus, it does not only aim to classify the given points but tries to learn the underlying probability distribution P(y|x) and, after training is completed, be able to classify correctly unseen points (hoping that they follow the same probability distribution). Therefore, the separating line is more likely to generalize to unseen data if it has a wider margin to the closest points in the training data.

As you may recall from High School, the equation of a line can be expressed as: $\boldsymbol{w} \cdot \boldsymbol{x}+b=0$, where $w$ is a vector normal to the line. We can also define the equation of a parallel line that passes through the closest point to the separating line, as well as the equation associated to the symmetric line. Normalizing these equations, they can be expressed as: $\boldsymbol{w} \cdot \boldsymbol{x}+b = \pm 1$, where the distance between them can be shown to be $2/||\boldsymbol{w}||$. Hence, the goal is to maximize this margin, i.e., solve the following optimization problem:

$$
\begin{align*}
\text{Minimize} \quad & \frac{1}{2} \| w\|^2, \\
\text{subject to} \quad & y_j \left( \vec{w} \cdot \vec{x}_j + b \right) \geq 1, \quad \forall j.
\end{align*}
$$

This optimization problem is known as the hard margin case, as no point is allowed to be misclassified or lie inside the margin. However, ML datasets usually contain noise and the goal of ML algorithms is to distinguish noise from important information. Thus, the previous constraint is relaxed, and the optimization problem that SVM actually solves is the following (we skip the derivation here, but it can be found in [4]):

$$
\begin{align*}
\text{Maximize} \quad & \sum_{j} \alpha_j - \frac{1}{2} \sum_{j,k} y_j y_k \alpha_j \alpha_k \left( \vec{x}_j \cdot \vec{x}_k \right), \\
\text{subject to} \quad & 0 \leq \alpha_j \leq C, \quad \forall j, \\
& \sum_{j} \alpha_j y_j = 0.
\end{align*}
$$

This is a convex quadratic programming problem, which can be solved using Sequential Minimal Optimization (SMO) [5]. Finally, we note that the SVM does not only apply to datasets with two features (or points in a plane), this was just done for visualization purposes, but can be applied to datasets with arbitrary number of features. Thus, in the general case, the separating line becomes an hyperplane, but the mathematics do not change.

### 2. Feature map

The algorithm defined so far can only classify data satisfactorily if it is linearly separable. This is rarely the case, especially for datasets with many features. There is a trick though, which consists on mapping the data to a higher dimensional space (known as feature map), and hope that in this space the data points are linearly separable (see Figure below). This procedure is known as the Kernel trick in the literature, as $\vec{x}_j \cdot \vec{x}_k $ is replaced by $K(\vec{x}_j, \vec{x}_k) = \langle \phi(\vec{x}_j)|\phi(\vec{x}_k) \rangle$, where $\phi$ denotes the feature map. Applying the kernel for all the possible pairs of datapoints defines the kernel matrix.

![feature map](figures_qsvm/feature%20map.png)


### 3. Quantum SVM

Having explained the classical SVM algorithm, we are ready to go Quantum. The idea is to use a parametrized quantum circuit as the feature map, and make use of the high dimensionality of the Hilbert space ($2^{n}$ where $n$ is the number of qubits). There are many different ways to map vectors to quantum states [3] but here we will explain and implement the angle encoding feature map. It consists of, starting with $n$ qubits (which must coincide with the number of features of the dataset) in state $|0 \rangle$, applying a rotation gate to each qubit, where the angle for each gate is the value of the corresponding feature.

Since what we need is the kernel matrix (composed by the inner product of vectors in feature space), we need to calculate the following: $K(\vec{x}_j, \vec{x}_k) = \langle \phi(\vec{x}_j),\phi(\vec{x}_k) \rangle = |\langle 0|\phi ^\dagger(\vec{x}_j)\phi(\vec{x}_k)|0\rangle |^2$. Hence, after applying the feature map to the first vector, we need to apply the inverse feature map (replacing every gate by its inverse) to the second vector. Finally, we will execute the quantum circuit $n_{shots}$ times (measuring in the computational basis) and estimate the probability of measuring all qubits in the $|0\rangle$ state.

Having calculated the quantum kernel matrix, which requires $n_{shots}\times n_{samples}^2$ runs of the quantum circuit (with number of qubits = number of features), we can apply the "normal" SVC and SMO to find the optimal classifying hyperplane. After the optimal normal vector and bias of the hyperplane is found, i.e. training is complete, the algorithm can predict the label of new data points by calculating: $sign(\omega^*x_{new} + b^*)$

### 4. Implementation

This example is inspired by the book "A Practical Guide to Quantum Machine Learning and Quantum Optimization" 
by Elías F.Combarro and Samuel González-Castillo. It uses a dataset for wine classification, which has the following characteristics (see https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html for more details).

<img src="figures_qsvm/wine_dataset.png" alt="Wine dataset" width="500">



0. Connect to QAAS if you want to use Qaptiva.

In [1]:
connect = False  # Set to False if you want to run locally without QLMaaS

if connect:
    from qat.qlmaas import QLMaaSConnection
    conn = QLMaaSConnection(hostname="qlm35e.neasqc.eu", check_host=False)

1. Import the necessary libraries and the self-made QSVM class.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA
from itertools import combinations
from tqdm.notebook import tqdm

from myqml import QSVM

2. Filter the wine dataset so that there are only two types of wines, yielding a binary classification problem.

In [3]:
x, y = load_wine(return_X_y=True)
x = x[:59+71,:]
y = y[:59+71]
print(f"Dimension of X: {x.shape}")
print(f"Dimension of y: {y.shape}")

Dimension of X: (130, 13)
Dimension of y: (130,)


3. Split the data into training and test sets so that the performance of the model is measured on unseen data, and scale the features using only the training data to prevent data leakage.

In [4]:
seed = 1
np.random.seed(seed)
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.8, random_state=seed)
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)
x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)
print(f"Scaled training inputs:\n{x_tr}")
print(f"\nDimension of training dataset: {x_tr.shape}")

Scaled training inputs:
[[0.81456507 0.25507901 0.77950311 ... 0.76608187 0.68       0.375     ]
 [0.92717465 0.39051919 0.7484472  ... 0.67251462 0.725      0.78571429]
 [0.88739042 0.53273138 0.82919255 ... 0.60233918 0.7925     0.70535714]
 ...
 [0.79501011 0.48081264 0.86335404 ... 0.56725146 0.61       0.27738095]
 [0.90964262 0.37471783 0.69565217 ... 0.57309942 0.695      0.28095238]
 [0.87997303 0.3724605  0.79192547 ... 0.65497076 0.6275     0.6577381 ]]

Dimension of training dataset: (104, 13)


4. Since the dataset has 13 features, the quantum circuits require 13 qubits. In order to reduce the number of qubits and thus, the computational complexity, we will perform a dimensionality reduction technique, termed as Principal Component Analysis (PCA). This method first finds the directions of maximum variance (principal components) and then performs a linear transformation, or change of basis, so that the feature space (rows of the dataset) is expressed in the basis of principal components. Finally, the columns associated with the directions of least variance are removed, as they contain the least amount of information. In this tuturial, you can choose the value of n_components, which determines the number of transformed features that are kept. For an introduction to PCA, we suggest reading [6]. 

In [ ]:
n_components = 5
pca = PCA(n_components = n_components)
xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

5. Define the quantum kernel using angle encoding or ZZ Feature map. For the two datasets studied in this tutorial, angle encoding performs generally better and is faster. Prove it yourself!

In [6]:
# Define the attributes of the QSVM algorithm
n_qubits = n_components
device = 'myQLM'
kernel_circuit_label = "angle_encoding" # 'angle encoding' or 'zz_encoding'
angle_encoding_type = "y" #'x', 'y' or 'z' for 'angle encoding' and any of them or None for 'zz_encoding'

# Create a QSVM instance 
qsvm = QSVM(n_qubits=n_qubits,
            device = device,
            kernel_circuit_label=kernel_circuit_label,
            angle_encoding_type=angle_encoding_type
            )

6. Calculate and print the kernel matrix associated with the training data points (optional).

In [7]:
print_kernel_matrix = False # Set to True if you want to print the kernel matrix
if print_kernel_matrix:
    kernel_matrix = qsvm.qkernel(xs_tr, xs_tr)
    print("Kernel matrix:\n", kernel_matrix)
    print("\nDimensions of the Kernel matrix: ", kernel_matrix.shape)

7. Fit the model to the training data.

In [8]:
qsvm.fit(xs_tr, y_tr)
print("QSVM model trained.")

QSVM model trained.


8. Predict the labels for the test set and calculate the accuracy of the model.

In [9]:
y_pred = qsvm.predict(xs_test)
print(f"Size of (transformed) test dataset: {xs_test.shape}")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of QSVM: {accuracy:.2f}")

Size of (transformed) test dataset: (26, 5)
Accuracy of QSVM: 0.96


5. Evaluate the confusion matrix, which shows: the true positives (top left), the false positives (top right), the false negatives (bottom left), and the true negatives (bottom right).

In [18]:
conf_mat = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n {conf_mat}")

Confusion matrix:
 [[72  0]
 [ 5 37]]


### 5. Use a different dataset: Breast Cancer Detection

We now employ a more difficult but still tractable dataset, which determines if a person has breast cancer depending on different characteristics. The details of the dataset are shown in the figure below, taken from the website: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html.

<img src="figures_qsvm/breast_cancer.png" alt="Breast cancer" width="500">


0. Import dataset

In [11]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
x = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

# Convert to numpy arrays 
x = x.to_numpy()
y = y.to_numpy()
  
# variable information 
print(breast_cancer_wisconsin_diagnostic.variables) 


                  name     role         type demographic description units  \
0                   ID       ID  Categorical        None        None  None   
1            Diagnosis   Target  Categorical        None        None  None   
2              radius1  Feature   Continuous        None        None  None   
3             texture1  Feature   Continuous        None        None  None   
4           perimeter1  Feature   Continuous        None        None  None   
5                area1  Feature   Continuous        None        None  None   
6          smoothness1  Feature   Continuous        None        None  None   
7         compactness1  Feature   Continuous        None        None  None   
8           concavity1  Feature   Continuous        None        None  None   
9      concave_points1  Feature   Continuous        None        None  None   
10           symmetry1  Feature   Continuous        None        None  None   
11  fractal_dimension1  Feature   Continuous        None        

1. Split training and test data, and apply scaling

In [12]:
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.8, random_state=seed)
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)
x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)

2. Apply PCA to reduce the number of features from 30 to 5

In [13]:
n_components = 5
pca = PCA(n_components = n_components)
xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

3. Use angle encoding and fit the training data using QSVM algorithm

In [14]:
n_qubits = n_components
device = 'myQLM'
kernel_circuit_label = "angle_encoding"
angle_encoding_type = "y"
qsvm = QSVM(n_qubits=n_qubits, 
            device=device, 
            kernel_circuit_label=kernel_circuit_label,
            angle_encoding_type=angle_encoding_type
            )
start = time.time()
qsvm.fit(xs_tr, y_tr)
end = time.time()
comp_time = end - start
print("Fitting completed in ", comp_time, ' s')

c:\Users\a943763\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:746: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


Fitting completed in  5115.112501382828  s


4. Predict the labels for the test set and calculate the accuracy of the model.

In [15]:
y_pred = qsvm.predict(xs_test)
print(f"Size of (transformed) test dataset: {xs_test.shape}")
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of QSVM: {accuracy:.2f}")

Size of (transformed) test dataset: (114, 5)
Accuracy of QSVM: 0.96


5. Evaluate the confusion matrix, which shows: the true positives (top left), the false positives (top right), the false negatives (bottom left), and the true negatives (bottom right).

In [17]:
conf_mat = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n {conf_mat}")

Confusion matrix:
 [[72  0]
 [ 5 37]]


In conclusion, we have shown how quantum computing can be integrated with a basic supervised ML algorithm to learn how to predict labels in datasets. Although we have achieved a very high accuracy for both datasets studied, it is important to note that real-life datasets are way more extensive and complex. Therefore, using this QSVM approach would require the execution of a vast number of quantum circuits of many qubits, which is intractable with current NISQ quantum computers and quantum emulators.

### 6. Advanced: Use your own custom quantum kernel for QSVM, defining it in myQLM or any other quantum programming language, such as Qiskit, Pennylane or Cirq.

Using overriding, you can define a child class of QSVM which defines a different kernel_circuit method using any quantum SDK. As an alternative, only compaible with myQLM, you can define a function that creates the quantum kernel quantum program, and input the function in the custom_kernel argument.

In [ ]:
class Custom_QSVM(QSVM):
    def kernel_circuit(self, a, b):
        pass

        '''
        TO BE FILLED BY THE USER 

        Thuis function should return a quantum program that computes the kernel between two vectors a and b, submit the corresonding job to the quantum simulator
        or QPU, and return the probability of measuring all qubits in the |0> state.
        '''

        return probability_all_zeros
    
def custom_kernel(a, b):

    '''
    TO BE FILLED BY THE USER (USE myQLM)

    This function should return a quantum program that computes the kernel between two vectors a and b.
    '''

    return qprogram

In [ ]:
method = 'overriding' # 'overriding' or 'custom_kernel'

if method == 'overriding':
    custom_qsvm = Custom_QSVM(n_qubits)
    custom_qsvm.fit(xs_tr, y_tr)
    y_pred = custom_qsvm.predict(xs_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Results using overriding custom kernel...")
    print(f"Accuracy of QSVM: {accuracy:.2f}\n")
    conf_mat = confusion_matrix(y_test, y_pred)
    print(f"Confusion matrix:\n {conf_mat}")

else:
    custom_qsvm = QSVM(n_qubits=n_qubits, 
            device=device, 
            custom_kernel=custom_kernel
            )
    custom_qsvm.fit(xs_tr, y_tr)
    y_pred = custom_qsvm.predict(xs_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Results using custom kernel function argument...")
    print(f"Accuracy of QSVM: {accuracy:.2f}\n")
    conf_mat = confusion_matrix(y_test, y_pred)
    print(f"Confusion matrix:\n {conf_mat}")

### 7. References
[1] Medium article that explains ML: https://medium.com/@RobuRishabh/introduction-to-machine-learning-555b0f1b62f5

[2] Bishop, C. M. (2006). *Pattern Recognition and Machine Learning*. Springer

[3] Schuld & Killoran, *A Practical Guide to Quantum Machine Learning and Quantum Optimization*

[4] Abu-Mostafa et al. *Learning from  Data*, e-Ch. 8

[5] Platt, J. (1998). *Sequential minimal optimization: A fast algorithm for training support vector machines*

[6] Shlens, J. (2014). *A tutorial on principal component analysis*. arXiv preprint arXiv:1404.1100.